In [1]:
import os
import json
import progressbar
import math
import pandas as pd
import numpy as np
from pandas import json_normalize
from functools import reduce

In [12]:
CITIES_FILEPATH = '../Dataset/Location/cities.json'
STATES_FILEPATH = '../Dataset/Location/states.json'
COUNTRIES_FILEPATH = '../Dataset/Location/countries.json'
LOCATION_FILEPATH = '../Dataset/Processed/Locations.csv'
USERS_FILEPATH = '../Dataset/BX-CSV-Dump/BX-Users.csv'
BOOKS_FILEPATH = '../Dataset/BX-CSV-Dump/BX-Books.csv'
USERS_NORMALIZED_FILEPATH = '../Dataset/Processed/BX-Users.csv'
LOC_FUZZY_MAP_FILEPATH = '../Dataset/Processed/mappings.json'
ISBN_FILEPATH = '../Dataset/Processed/isbn.json'

In [19]:
books = pd.read_csv(BOOKS_FILEPATH, sep=';', encoding='Latin')

In [21]:
isbn = pd.read_json(ISBN_FILEPATH)

In [22]:
isbn

,ISBN
0,0195153448
1,0002005018
10,0771074670
100,0385235941
1000,0064472779
...,...
99995,3499221527
99996,0671653911
99997,1565970616
99998,3442726832


In [23]:
books.head(11)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...
7,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...
8,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,http://images.amazon.com/images/P/0679425608.0...,http://images.amazon.com/images/P/0679425608.0...,http://images.amazon.com/images/P/0679425608.0...
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...,http://images.amazon.com/images/P/074322678X.0...


In [13]:
books[['ISBN']].to_json(ISBN_FILEPATH)

In [ ]:
def sanitizeNumericData(row, as_types):
    for col in as_types:
        row[col] = int(''.join(filter(str.isdigit, row[col]))) if as_types[col] == 'int64' else row[col]
    return row

In [ ]:
def loadDataframeFromJSON(filepath, key, rename_columns, as_types={}):
    with open(filepath) as f: 
        d = json.load(f)  
    return json_normalize(d[key]).apply(sanitizeNumericData, args=(as_types, ), axis=1).rename(columns=rename_columns)

In [3]:
if not os.path.exists(LOCATION_FILEPATH):
    cities = loadDataframeFromJSON(CITIES_FILEPATH, 'cities', {'id': 'city_id', 'name': 'city'}, {'id': 'int64', 'state_id': 'int64'})
    states = loadDataframeFromJSON(STATES_FILEPATH, 'states', {'id': 'state_id', 'name': 'state'}, {'id': 'int64','country_id': 'int64'})
    countries = loadDataframeFromJSON(COUNTRIES_FILEPATH, 'countries', {'id': 'country_id', 'sortname': 'country_code', 'phoneCode': 'phone_code', 'name': 'country'})
    locations = cities.merge(states, on='state_id', how="inner").merge(countries, on='country_id', how="inner")
    locations.to_csv(LOCATION_FILEPATH, index=False, index_label=False, columns=['city_id', 'city', 'state_id', 'state', 'country_id', 'country', 'country_code', 'phone_code'])
else:
    locations = pd.read_csv(LOCATION_FILEPATH);

In [4]:
locations['Location'] = locations['city'] + ", " + locations['state'] + ", " + locations['country']

In [5]:
locations.head()

,city_id,city,state_id,state,country_id,country,country_code,phone_code,Location
0,1,Bombuflat,1,Andaman and Nicobar Islands,101,India,IN,91,"Bombuflat, Andaman and Nicobar Islands, India"
1,2,Garacharma,1,Andaman and Nicobar Islands,101,India,IN,91,"Garacharma, Andaman and Nicobar Islands, India"
2,3,Port Blair,1,Andaman and Nicobar Islands,101,India,IN,91,"Port Blair, Andaman and Nicobar Islands, India"
3,4,Rangat,1,Andaman and Nicobar Islands,101,India,IN,91,"Rangat, Andaman and Nicobar Islands, India"
4,5,Addanki,2,Andhra Pradesh,101,India,IN,91,"Addanki, Andhra Pradesh, India"


In [26]:
def loadNormalizedUsers():
    if not os.path.exists(USERS_NORMALIZED_FILEPATH):
        users = pd.read_csv(USERS_FILEPATH, sep=";")
        users_loc = pd.DataFrame([ x.split(', ', 2)[0:3] for x in users['Location'].tolist() ])
        users_loc.columns = ['city', 'state', 'country']
        users_normalized = pd.concat([users, users_loc], axis=1)
        users_normalized['fuzzy_match'] = np.nan
    else:
        users_normalized = pd.read_csv(USERS_NORMALIZED_FILEPATH, dtype={'fuzzy_match': str, "city": str, "state": str})
    return users_normalized

users_normalized = loadNormalizedUsers()
users_normalized.loc[120:140]

In [8]:
users_normalized.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278770 entries, 0 to 278857
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   User-ID      278770 non-null  int64  
 1   Location     278770 non-null  object 
 2   Age          168048 non-null  float64
 3   city         278210 non-null  object 
 4   state        265613 non-null  object 
 5   country      274178 non-null  object 
 6   fuzzy_match  0 non-null       object 
dtypes: float64(1), int64(1), object(5)
memory usage: 17.0+ MB


In [ ]:
# ! conda install -c conda-forge fuzzywuzzy

In [9]:
from fuzzywuzzy import fuzz, process

In [10]:
def getPotentialMatch(city, state):
    booleans = []
    for row in locations.itertuples():
        if fuzz.WRatio(city, row.city) > 90:
            booleans.append(True)
        else:
            booleans.append(False)
    return booleans

In [24]:
f = open(LOC_FUZZY_MAP_FILEPATH,)  
mapping = json.load(f)
def fuzzy_match_loc(row):
    index = row.city + "|" + row.state
    if(index not in mapping):
        filters = getPotentialMatch(row.city, row.state);
        potential_matches = locations[filters]
        if potential_matches['Location'].count() > 0:
            row.fuzzy_match = process.extractOne(row.Location, list(potential_matches['Location'].values))[0]
            mapping[index] = row.fuzzy_match
    else:
       row.fuzzy_match  = mapping[index]
    return row

In [57]:
n = 1000
chunk=0
stop_limit = 20
users_normalized = loadNormalizedUsers()
to_process_users= users_normalized[users_normalized['fuzzy_match'].isna()].fillna('')
total_chunk = to_process_users.shape[0] // n
list_df = [to_process_users[i:i+n] for i in range(0,to_process_users.shape[0],n)]
for df in progressbar.progressbar(list_df, redirect_stdout=True):
    updated_df = df.apply(fuzzy_match_loc, axis=1)
    users_normalized.loc[users_normalized['User-ID'].isin(updated_df['User-ID']), ['fuzzy_match']] = updated_df[['fuzzy_match']]
    users_normalized.to_csv(USERS_NORMALIZED_FILEPATH, index=False, index_label=False)
    saveLocMapping()
    chunk=chunk+1
    if chunk == stop_limit:
        updated_df.head(20)
        break

  7% (19 of 264) |#              | Elapsed Time: 6:17:47 ETA:  2 days, 12:45:50

In [25]:
len(mapping)

47008

In [46]:
def saveLocMapping():
    json_content = json.dumps(mapping)
    f = open(LOC_FUZZY_MAP_FILEPATH,"w")
    f.write(json_content)
    f.close()

In [28]:
len(users_normalized['Location'])

278770

In [30]:
users_normalized = loadNormalizedUsers()
users_normalized[users_normalized['fuzzy_match'].isna()]

,User-ID,Location,Age,city,state,country,fuzzy_match
17,18,"rio de janeiro, rio de janeiro, brazil",25.0,rio de janeiro,rio de janeiro,brazil,NaN
64,65,"n/a, n/a, australia",NaN,NaN,NaN,australia,NaN
90,91,"toronto/newmarket, ,",NaN,toronto/newmarket,",",NaN,NaN
111,112,"mexico city, d.f., mexico",32.0,mexico city,d.f.,mexico,NaN
155,156,"sarajevo, n/a, bosnia and herzegovina",20.0,sarajevo,NaN,bosnia and herzegovina,NaN
...,...,...,...,...,...,...,...
278607,278608,"west vancouver, british columbia, canada",NaN,west vancouver,british columbia,canada,NaN
278700,278701,"vila das aves, minho, portugal",39.0,vila das aves,minho,portugal,NaN
278711,278712,"cranbrook, british columbia, canada",NaN,cranbrook,british columbia,canada,NaN
278781,278782,"larnaca, n/a, cyprus",NaN,larnaca,NaN,cyprus,NaN
